In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42): 
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [6]:
learning_rate = 0.001
batch_size = 16
train_size = X_train2.shape[0]

#saver = tf.train.Saver()

with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('/Team48_HW2.ckpt.meta', clear_devices=True)
    new_saver.restore(sess, '/Team48_HW2.ckpt')
    xs = tf.get_default_graph().get_tensor_by_name("xs:0")
    ys = tf.get_default_graph().get_tensor_by_name("ys:0")
    y = tf.get_default_graph().get_tensor_by_name("y:0")
    #logits = y.op.inputs[0]
    accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
    loss = tf.get_default_graph().get_tensor_by_name("loss:0")
    optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars, name="training_op")
    
    tf.global_variables_initializer().run() 
    
    best_loss = 99999
    stand_epoch = 0
    
    for i in range(1000):
        #sess.run(training_op, feed_dict={xs: X_train2, ys: y_train2})
        stand_epoch = stand_epoch + 1
        for j in range(int(train_size/batch_size)):
            # get batch data each time
            batch_x = X_train2[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(training_op, feed_dict={xs: batch_x, ys: batch_y})
        
        val_loss = sess.run(loss, feed_dict={xs: X_valid2, ys: y_valid2})
        if best_loss > val_loss:
            stand_epoch = 0
            best_loss = val_loss
            
        
        accu = sess.run(accuracy, feed_dict={xs: X_test2, ys: y_test2})
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu)
        
        if stand_epoch >= 20: # Early stop
            print("Early stop in epoch", i)
            break
    save_path = new_saver.save(sess, "/Team48_HW3_1.ckpt") # save variables to checkpoint
    sess.close()

    print("Training Finish!!!")

INFO:tensorflow:Restoring parameters from /Team48_HW2.ckpt
0 Validation loss: 233.408  ; Best loss: 233.408  ; Accuracy: 0.310019
1 Validation loss: 227.191  ; Best loss: 227.191  ; Accuracy: 0.397243
2 Validation loss: 221.199  ; Best loss: 221.199  ; Accuracy: 0.465542
3 Validation loss: 216.1  ; Best loss: 216.1  ; Accuracy: 0.507303
4 Validation loss: 211.461  ; Best loss: 211.461  ; Accuracy: 0.550298
5 Validation loss: 206.687  ; Best loss: 206.687  ; Accuracy: 0.590208
6 Validation loss: 201.627  ; Best loss: 201.627  ; Accuracy: 0.624151
7 Validation loss: 197.194  ; Best loss: 197.194  ; Accuracy: 0.651718
8 Validation loss: 193.918  ; Best loss: 193.918  ; Accuracy: 0.668998
9 Validation loss: 191.355  ; Best loss: 191.355  ; Accuracy: 0.679901
10 Validation loss: 189.202  ; Best loss: 189.202  ; Accuracy: 0.687307
11 Validation loss: 187.352  ; Best loss: 187.352  ; Accuracy: 0.698416
12 Validation loss: 185.75  ; Best loss: 185.75  ; Accuracy: 0.70541
13 Validation loss: 18

115 Validation loss: 166.559  ; Best loss: 166.559  ; Accuracy: 0.802304
116 Validation loss: 166.53  ; Best loss: 166.53  ; Accuracy: 0.80251
117 Validation loss: 166.501  ; Best loss: 166.501  ; Accuracy: 0.802715
118 Validation loss: 166.472  ; Best loss: 166.472  ; Accuracy: 0.802921
119 Validation loss: 166.443  ; Best loss: 166.443  ; Accuracy: 0.80251
120 Validation loss: 166.414  ; Best loss: 166.414  ; Accuracy: 0.803333
121 Validation loss: 166.384  ; Best loss: 166.384  ; Accuracy: 0.803744
122 Validation loss: 166.354  ; Best loss: 166.354  ; Accuracy: 0.804155
123 Validation loss: 166.324  ; Best loss: 166.324  ; Accuracy: 0.804567
124 Validation loss: 166.293  ; Best loss: 166.293  ; Accuracy: 0.804155
125 Validation loss: 166.262  ; Best loss: 166.262  ; Accuracy: 0.804567
126 Validation loss: 166.23  ; Best loss: 166.23  ; Accuracy: 0.804773
127 Validation loss: 166.197  ; Best loss: 166.197  ; Accuracy: 0.804978
128 Validation loss: 166.164  ; Best loss: 166.164  ; Acc

230 Validation loss: 163.717  ; Best loss: 163.717  ; Accuracy: 0.827196
231 Validation loss: 163.675  ; Best loss: 163.675  ; Accuracy: 0.827196
232 Validation loss: 163.633  ; Best loss: 163.633  ; Accuracy: 0.827402
233 Validation loss: 163.59  ; Best loss: 163.59  ; Accuracy: 0.827607
234 Validation loss: 163.547  ; Best loss: 163.547  ; Accuracy: 0.82843
235 Validation loss: 163.504  ; Best loss: 163.504  ; Accuracy: 0.828842
236 Validation loss: 163.461  ; Best loss: 163.461  ; Accuracy: 0.828842
237 Validation loss: 163.417  ; Best loss: 163.417  ; Accuracy: 0.829665
238 Validation loss: 163.374  ; Best loss: 163.374  ; Accuracy: 0.830487
239 Validation loss: 163.331  ; Best loss: 163.331  ; Accuracy: 0.831516
240 Validation loss: 163.288  ; Best loss: 163.288  ; Accuracy: 0.831516
241 Validation loss: 163.245  ; Best loss: 163.245  ; Accuracy: 0.832133
242 Validation loss: 163.204  ; Best loss: 163.204  ; Accuracy: 0.832545
243 Validation loss: 163.164  ; Best loss: 163.164  ; 